# Toronto Clustering

#### Importing needed libraries:

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

#!conda install -c conda-forge geocoder --yes
import geocoder

print('Libraries imported.')

Libraries imported.


## 1. Download the file and get it into the wanted Dataframe form:

In [2]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
df = df[0]

In [3]:
print(df.shape)
df.head(10)

(290, 3)


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


#### Drop cells with a borough that is Not assigned:

In [4]:
df = df[df[1]!='Not assigned']
df.head(15)

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


Now the shape of the Dataframe is:

In [5]:
df.shape

(213, 3)

So 77 raws deleted.

#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. (see line 9):

In [6]:
df.iloc[7,2]=df.iloc[7,1] #(index=7)
df.head(10)

,0,1,2
0,Postcode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge


#### Reseting index:

In [7]:
df.reset_index(inplace=True)
df.head()

,index,0,1,2
0,0,Postcode,Borough,Neighbourhood
1,3,M3A,North York,Parkwoods
2,4,M4A,North York,Victoria Village
3,5,M5A,Downtown Toronto,Harbourfront
4,6,M5A,Downtown Toronto,Regent Park


In [8]:
df.drop('index',axis=1, inplace=True)
df.head(6)

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,Harbourfront
4,M5A,Downtown Toronto,Regent Park
5,M6A,North York,Lawrence Heights


#### More than one neighborhood can exist in one postal code area:

In [10]:
df1 = df[df[0].duplicated()==True].reset_index() #duplicate dataframe
df1.head()

,index,0,1,2
0,4,M5A,Downtown Toronto,Regent Park
1,6,M6A,North York,Lawrence Manor
2,10,M1B,Scarborough,Malvern
3,13,M4B,East York,Parkview Hill
4,15,M5B,Downtown Toronto,Garden District


In [11]:
s=0
for i in df1.iloc[:,0].tolist():
    for j in range(i):
        if df.iloc[j,0]==df1.iloc[s,1]:
            df.iloc[j,2]=df.iloc[j,2]+','+df1.iloc[s,3]
    s=s+1

In [12]:
df.drop(df1['index'].tolist(),axis=0,inplace=True)
df.head(10)

,0,1,2
0,Postcode,Borough,Neighbourhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront,Regent Park"
5,M6A,North York,"Lawrence Heights,Lawrence Manor"
7,M7A,Queen's Park,Queen's Park
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,"Rouge,Malvern"
11,M3B,North York,Don Mills North
12,M4B,East York,"Woodbine Gardens,Parkview Hill"


In [13]:
df.shape

(104, 3)

#### Get it in the final form:

In [14]:
df.rename(columns={0:'PostalCode', 1:'Borough', 2:'Neighborhood'},inplace=True) #rename columns
df.drop(index=0,inplace=True) #drop column raw
df.reset_index(inplace=True) #reset index
df.drop('index',axis=1,inplace=True) #drop index column
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [18]:
print(df.shape)
df.tail()

(103, 3)


,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre969 Eastern
101,M8Y,Etobicoke,"Humber Bay,King's Mill Park,Kingsway Park Sout..."
102,M8Z,Etobicoke,"Kingsway Park South West,Mimico NW,The Queensw..."


#### Let's find the latitude and lonitude of postal codes:

We can do that using geocoder but after the changes of the instructor we have a new excel file.
1st Way:

In [37]:
#list_lat=[]
#list_lng=[]

# initialize your variable to None
#lat_lng_coords = None

#for i in range(103):
#    postal_code=df.iloc[i,0]
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng
#    list_lat.append(lat_lng_coords[0])
#    list_lng.append(lat_lng_coords[1])
#    lat_lng_coords = None

#### Add lists to the Dataframe and take the final form:

In [80]:
#df_lat=pd.DataFrame(list_lat,columns={'Latitude'})
#df_lng=pd.DataFrame(list_lng,columns={'Longitude'})

In [222]:
#neighborhoods = pd.concat([df, df_lat,df_lng], axis=1)

In [228]:
#print(neighborhoods.shape)
#neighborhoods.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


2nd Way:

In [43]:
df_lat_lng = pd.read_csv("C:/Users/Sotiris/Desktop/python works/applied data science specialization/Applied data science cupstone/Geospatial_Coordinates (1).csv", sep=',')
print(df_lat_lng.shape)
df_lat_lng.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [44]:
column_names=['Latitude', 'Longitude']
df_ll=pd.DataFrame(columns=column_names)
df_ll

,Latitude,Longitude


In [45]:
for i in range(103):
    for j in range(103):
        if df.iloc[i,0]==df_lat_lng.iloc[j,0]:
            df_ll=df_ll.append({'Latitude':df_lat_lng.iloc[j,1],
                              'Longitude':df_lat_lng.iloc[j,2]}, ignore_index=True)
            

In [57]:
neighborhoods=pd.concat([df, df_ll], axis=1)

In [62]:
print(neighborhoods.shape)
neighborhoods.head(10)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens,Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937


And we take a DataFrame same as in 1st Way, as we should.

#### Get only Boroughs that contain the word Toronto:

In [59]:
df_tor=neighborhoods
for i in range(df_tor.shape[0]):
    if 'Toronto' not in df_tor.iloc[i,1]:
        df_tor.iloc[i,1] = np.nan
neighborhoods=pd.concat([df, df_ll], axis=1) #set neighborhoods dataframe as before
df_tor.head(7)        

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,NaN,Parkwoods,43.753259,-79.329656
1,M4A,NaN,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,NaN,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,NaN,Queen's Park,43.662301,-79.389494
5,M9A,NaN,Islington Avenue,43.667856,-79.532242
6,M1B,NaN,"Rouge,Malvern",43.806686,-79.194353


In [60]:
df_tor = df_tor.dropna(axis=0)
df_tor.reset_index(inplace=True)
df_tor.drop('index', axis=1, inplace=True)

In [61]:
print(df_tor.shape)
df_tor.head(7)

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564


#### Get geographical coordinates of Toronto:

In [63]:
address = 'Toronto'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

C:\Users\Sotiris\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Map the Neigborhoods of Toronto:

In [64]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [65]:
# create map of Toronto (Borough contains the word Toronto) using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    if 'Toronto' in borough:     #key code difference in comparison with the above code cell
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

In [121]:
#I just saved the dataframe neighborhoods file in my local directory
#neighborhoods.to_csv('/Users/Sotiris/Desktop/toronto.csv', index=False)

## 2. Explore Neighborhoods in Toronto.

We are going to Work with df_tor data (Borough contains the word Toronto).

#### Define Foursquare Credentials and Version

In [66]:
LIMIT=100
CLIENT_ID = 'IAMCBNDMUZCPZAWW5ITR3B312HGB4CCECCHWO5IVA4V2PTSI' # your Foursquare ID
CLIENT_SECRET = 'QKOKZNOL3R0I2T0K3ZTDZFNCTWIKCIU1A0PS15HTXYW4VNDM' # your Foursquare Secret
#VERSION = '20180604'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IAMCBNDMUZCPZAWW5ITR3B312HGB4CCECCHWO5IVA4V2PTSI
CLIENT_SECRET:QKOKZNOL3R0I2T0K3ZTDZFNCTWIKCIU1A0PS15HTXYW4VNDM


In [67]:
#defined a little diffent as in the NY lab, i wanted Borough inside the returned dataframe
def getNearbyVenues(names, bors, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, bor, lat, lng in zip(names, bors, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            bor,
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough',
                             'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [73]:
toronto_venues = getNearbyVenues(names=df_tor['Neighborhood'],
                                 bors=df_tor['Borough'],
                                   latitudes=df_tor['Latitude'],
                                   longitudes=df_tor['Longitude']
                                  )

Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
The Danforth West,Riverdale
Design Exchange,Toronto Dominion Centre
Brockton,Exhibition Place,Parkdale Village
The Beaches West,India Bazaar
Commerce Court,Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North,Forest Hill West
High Park,The Junction South
North Toronto West
The Annex,North Midtown,Yorkville
Parkdale,Roncesvalles
Davisville
Harbord,University of Toronto
Runnymede,Swansea
Moore Park,Summerhill East
Chinatown,Grange Park,Kensington Market
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown,St. James Town
First Canadian Place,Underground city


In [74]:
print(toronto_venues.shape)
toronto_venues.head(10)

(1711, 8)


,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
3,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot
4,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Cooper Koo YMCA,43.653191,-79.357947,Gym / Fitness Center
5,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
6,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
7,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Dominion Pub and Kitchen,43.656919,-79.358967,Pub
8,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,Corktown Common,43.655618,-79.356211,Park
9,Downtown Toronto,"Harbourfront,Regent Park",43.65426,-79.360636,The Distillery Historic District,43.650244,-79.359323,Historic Site


I noticed that there are some venues that are neighborhoods in the above dataframe.

In [76]:
j=0
for cat in toronto_venues['Venue Category']:
    if cat=='Neighborhood':
        j+=1
print(j)

4


In [77]:
toronto_venues[toronto_venues['Venue Category']=='Neighborhood']

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
252,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
418,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
525,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
983,East Toronto,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


Lets drop that data:

In [81]:
toronto_venues = toronto_venues[toronto_venues['Venue Category']!='Neighborhood']
toronto_venues.reset_index(inplace=True)
toronto_venues.drop('index', axis=1, inplace=True)
print(toronto_venues.shape)

(1707, 8)


In [83]:
toronto_venues.tail()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1702,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,Toronto Yoga Mamas,43.664824,-79.324335,Yoga Studio
1703,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,Olliffe On Queen,43.664503,-79.324768,Butcher
1704,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,TTC Stop #03049,43.664470,-79.325145,Light Rail Station
1705,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
1706,East Toronto,Business reply mail Processing Centre969 Eastern,43.662744,-79.321558,Revolution Recording,43.662561,-79.326940,Recording Studio


Let's check how many venues were returned for each neighborhood:

In [84]:
toronto_venues.groupby('Neighborhood').count()

,Borough,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,,
"Adelaide,King,Richmond",99,99,99,99,99,99,99
Berczy Park,55,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22,22
Business reply mail Processing Centre969 Eastern,18,18,18,18,18,18,18
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",14,14,14,14,14,14,14
"Cabbagetown,St. James Town",52,52,52,52,52,52,52
Central Bay Street,83,83,83,83,83,83,83
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100,100
Christie,15,15,15,15,15,15,15


## 3. Analyse each Neighborhood

In [85]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront,Regent Park",0,0,0,0,0,0,0,0,0,0,0,0,0,

In [88]:
toronto_onehot.shape

(1707, 240)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [89]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Women's Store,Yoga Studio
0,"Adelaide,King,Richmond",0.010101,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040404,0.000000,0.000000,0.00,0.010101,0.010101,0.000000,0.020202,0.000000,0.000000,0.000000,0.000000,0.020202,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010101,0.000000,0.010101,0.020202,0.000000,0.000000,0.010101,0.030303,0.010101,0.00,0.000000,0.000000,0.060606,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020202,0.010101,0.060606,0.000000,0.000000,0.00,0.000000,0.

#### let's print each neighborhood along with the top 10 most common venues and put that into a Dataframe:

First define fuction

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the Dataframe

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Café,Coffee Shop,Thai Restaurant,American Restaurant,Steakhouse,Burger Joint,Restaurant,Bar,Gym,Hotel
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Restaurant,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Steakhouse,Bakery
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Breakfast Spot,Café,Gym,Climbing Gym,Convenience Store,Burrito Place,Stadium,Bar,Caribbean Restaurant
3,Business reply mail Processing Centre969 Eastern,Light Rail Station,Yoga Studio,Skate Park,Smoke Shop,Farmers Market,Fast Food Restaurant,Spa,Brewery,Burrito Place,Butcher
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Harbor / Marina,Airport Gate,Airport Food Court,Airport
5,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Pub,Café,Italian Restaurant,Pizza Place,Indian Restaurant,Bakery,Park,Gourmet Shop
6,Central Bay Street,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Thai Restaurant,Salad Place
7,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Bar,Chinese Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bakery,Coffee Shop,Dumpling Restaurant,Hotpot Restaurant
8,Christie,Grocery Store,Café,Park,Diner,Convenience Store,Coffee Shop,Italian Restaurant,Restaurant,Baby Store,Nightclub
9,Church and Wellesley,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Burger Joint,Gay Bar,Restaurant,Gastropub,Bubble Tea Shop,Men's Store,Café


## 4. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [94]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:30] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       4, 0, 1, 0, 0, 1, 3, 0])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [96]:
toronto_merged = df_tor

# add clustering labels
toronto_merged['Cluster Labels'] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636,0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Mexican Restaurant,Yoga Studio,Shoe Store,Brewery
1,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Tea Room,Pizza Place,Bubble Tea Shop,Diner
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Gastropub,Bakery,Cosmetics Shop,Japanese Restaurant,Italian Restaurant
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Park,Pub,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Café,Restaurant,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Steakhouse,Bakery


Finally, let's visualize the resulting clusters

In [97]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

## Cluster 1

In [108]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Harbourfront,Regent Park",0,Coffee Shop,Park,Bakery,Café,Breakfast Spot,Pub,Mexican Restaurant,Yoga Studio,Shoe Store,Brewery
1,"Ryerson,Garden District",0,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Japanese Restaurant,Bar,Tea Room,Pizza Place,Bubble Tea Shop,Diner
2,St. James Town,0,Coffee Shop,Restaurant,Café,Hotel,Clothing Store,Gastropub,Bakery,Cosmetics Shop,Japanese Restaurant,Italian Restaurant
3,The Beaches,0,Coffee Shop,Park,Pub,Yoga Studio,Dim Sum Restaurant,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
4,Berczy Park,0,Coffee Shop,Cocktail Bar,Café,Restaurant,Cheese Shop,Farmers Market,Seafood Restaurant,Beer Bar,Steakhouse,Bakery
5,Central Bay Street,0,Coffee Shop,Café,Italian Restaurant,Japanese Restaurant,Sandwich Place,Bubble Tea Shop,Bar,Burger Joint,Thai Restaurant,Salad Place
6,Christie,0,Grocery Store,Café,Park,Diner,Convenience Store,Coffee Shop,Italian Restaurant,Restaurant,Baby Store,Nightclub
7,"Adelaide,King,Richmond",0,Café,Coffee Shop,Thai Restaurant,American Restaurant,Steakhouse,Burger Joint,Restaurant,Bar,Gym,Hotel
8,"Dovercourt Village,Dufferin",0,Bakery,Supermarket,Fast Food Restaurant,Portuguese Restaurant,Smoke Shop,Brewery,Discount Store,Bar,Bank,Café
9,"Harbourfront East,Toronto Islands,Union Station",0,Coffee Shop,Hotel,Café,Aquarium,Pizza Place,Italian Restaurant,Scenic Lookout,Sports Bar,Brewery,Sushi Restaurant


## Cluster 2

In [109]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Lawrence Park,1,Dim Sum Restaurant,Park,Swim School,Bus Line,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
24,"Parkdale,Roncesvalles",1,Coffee Shop,Gift Shop,Piano Bar,Dessert Shop,Breakfast Spot,Eastern European Restaurant,Bar,Burger Joint,Dog Run,Movie Theater
27,"Runnymede,Swansea",1,Coffee Shop,Café,Sushi Restaurant,Pizza Place,Italian Restaurant,Baby Store,Indie Movie Theater,Bank,Bar,Diner


## Cluster 3

In [110]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
35,"First Canadian Place,Underground city",2,Coffee Shop,Café,Hotel,Restaurant,Steakhouse,American Restaurant,Gastropub,Deli / Bodega,Gym,Burger Joint


## Cluster 4

In [111]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,"Moore Park,Summerhill East",3,Trail,Park,Restaurant,Intersection,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dim Sum Restaurant,Dumpling Restaurant


## Cluster 5

In [112]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,North Toronto West,4,Coffee Shop,Sporting Goods Shop,Clothing Store,Gym / Fitness Center,Fast Food Restaurant,Metro Station,Mexican Restaurant,Diner,Dessert Shop,Park
